In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# from scipy.signal import argrelextrema

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Senzmate/RCA/jsonoutput3.csv",  on_bad_lines='skip')#reading csv file
df.head(10)#first 10 rows

In [ ]:
df.deviceId.unique()#checking how many devices are available

In [ ]:
print(len(df.deviceId.unique())) #number of devices available

294


In [ ]:
gp = df.groupby('deviceId')
print(type(gp))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [ ]:
gp.get_group('869170034808981')

In [ ]:
dataframes = [d for i, d in df.groupby('deviceId')]# grouping data of each device by deviceid and creating dataframes


In [ ]:
dataframes_new = []
names_new = []
dict_new = {}
for i, d in gp:
  dataframes_new.append(d)
  names_new.append(i)
  dict_new[i] = d

In [ ]:
dataframes[0]

In [ ]:
print(len(dataframes)) #number of dataframes created which is equal to number of devices available

In [ ]:
names = [i for i, d in df.groupby('deviceId')] #names of the devices are stored in a list

In [ ]:
print(names)

In [ ]:
names.index(67023401626)

##Functions

In [ ]:
data =[]
for i in range (len(dataframes)):
  print(names[i],(len(dataframes[i])))
  data.append(len(dataframes[i]))

print(max(data))

indx = data.index(max(data))
print(names[indx])

In [ ]:
def get_device_time(dfs,i):
  tn=[_ for _, d in dfs[i].groupby('time')]
  td=[d for _, d in dfs[i].groupby('time')]
  print(len(tn))
  return tn,td

In [ ]:
def create_dataframe(tn):
  return pd.DataFrame({'time':tn})

In [ ]:
def get_feature_dataframes(td,i):
  ft=[d for _, d in td[i].groupby('code')]
  return ft

In [ ]:
def get_feature_names(td,i):
  name=[_ for _, d in td[i].groupby('code')]
  return name

In [ ]:
def check_time(df_,x):
  if(x.time.isin(df_.time).any()):
    return x.value

In [ ]:
def add_features_to_dataframe(df_,ft,codenames,tn,i):
  for j in range(len(ft)):
    ft[j]=ft[j].reset_index()
    df_.loc[df_.time==tn[i],codenames[j]]=check_time(df_,ft[j]).values[0]
  return df_

In [ ]:
def create_csv_file(df,names,i):
  df.to_csv("/content/drive/MyDrive/RCA/devices/"+str(names[i])+".csv",index = False)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
def plot_chart(df,ft):
  matplotlib.rcParams['figure.figsize'] = (8,5)
  plt.plot(df["level_0"],df[ft],color='blue')
  plt.xlabel("Date&Time")
  plt.ylabel(ft)
  plt.title("Variation of "+ft+" with Date&Time")

In [ ]:
def find_outliers(dfs,ft):
  Q1 = df_[ft].quantile(0.25)
  Q3 = df_[ft].quantile(0.75)
  IQR = Q3 - Q1    #IQR is interquartile range. 
  # lower_bound=df_['B'] >= Q1 - 1.5 * IQR
  # upper_bound=df_['B'] <= Q3 + 1.5 *IQR
  outliers = df_[(df_[ft] <= Q1 - 1.5 * IQR) & (df_[ft] >= Q3 + 1.5 *IQR)]
  return outliers

##Automated Analysis

In [ ]:
def create_device_data(names,dataframes,num):
  print("Device ID: " ,names[num])
  print("Device sensor data count: \n")
  print(dataframes[num].number.value_counts())
  print("Device data shape: ",dataframes[num].shape)
  tn,td=get_device_time(dataframes,num)
  df_=create_dataframe(tn)
  df_=df_.drop_duplicates(subset=['time'])#keep first duplicate
  # print(df_.time.value_counts())
  # print(df_.shape)
  # df_=df_.reset_index()
  for i in range(len(tn)):
    factor=get_feature_dataframes(td,i)
    code_names=get_feature_names(td,i)
    df_=add_features_to_dataframe(df_,factor,code_names,tn,i)
  print("Device data shape: ",df_.shape)
  
  list_of_cols=['index','time','B','IT','LIA1','H','IRO','T','ST','SS']
  col_exists = [col for col in df_.columns if col not in list_of_cols]
  print("Dropped columns: ",col_exists)
  df_.drop(col_exists, axis='columns',inplace=True)
  print("Device data sample: \n")
  print(df_.head())
  col_names={'B':'Battery','IT':'Internal Temperature','LIA1':'Light Intensity','H':'Humidity','IRO':'Irometer','T':'Temperature','ST':'Soil Temperature','SS':'Signal Strength'}
 
  for c in df_.columns[1:]:
    df_=df_.rename(columns={c:col_names[c]})
  create_csv_file(df_,names,num)
  print("Device data sample: \n")
  print(df_.head())
  df_.reset_index(inplace=True)
  return df_




In [ ]:
def auto_analyse(names,num,df_):
  num_of_sensors=len(df_.columns)-1
  print("No of sensors: ",num_of_sensors)
  device_faults=pd.DataFrame()
  device_variations=pd.DataFrame()
  
  #Analysing null values
  print("Null column counts: \n")
  print(df_.isnull().sum())
  null_cols=df_.columns[df_.isnull().any()].tolist()
  col_names={'Battery':'B','Internal Temperature':'IT','Light Intensity':'LIA1','Humidity':'H','Irometer':'IRO','Temperature':'T','Soil Temperature':'ST','Signal Strength':'SS'}
  for col in null_cols:
    print("Time durations for ",col," null values: ")
    device_faults=device_faults.append(df_[df_[col].isnull()])
    device_faults.loc[device_faults['time'].isin(df_[df_[col].isnull()].time),'{}sensor'.format(col_names[col])]= col
    device_faults.loc[device_faults['time'].isin(df_[df_[col].isnull()].time),'{}Faults'.format(col_names[col])]='{}NULL'.format(col_names[col])
    print(df_[df_[col].isnull()])
  cols={'Battery':{'min':260,'max':314},'Internal Temperature':{'min':-55,'max':125},'Light Intensity':{'min':0,'max':65535},'Humidity':{'min':0,'max':110},'Irometer':{'min':0,'max':200},'Temperature':{'min':-40,'max':125},'Soil Temperature':{'min':-55,'max':125},'Signal Strength':{'min':2,'max':30}}
  
  if(len(df_.columns)>=2):
    for c in df_.columns[2:]:
      try:
        df_[c] = df_[c].apply(lambda x: float(x))
      except:
        df_[c] = df_[c].apply(lambda x: None)
        
      #Analysing wrong data points
      vals=[]
      for col in cols:
        if(col==c):
          for sen,val in cols[col].items():
            vals.append(val)
        else:
          pass
      print(c,vals)
      if((df_[c].min()<vals[0])|(df_[c].max()>vals[1])):
        print(c)
        print("Time durations for ",c," wrong data points: ")
        print(df_[(df_[c] < vals[0]) | (df_[c] > vals[1])])
        device_faults=device_faults.append(df_[(df_[c] < vals[0]) | (df_[c] > vals[1])])
        device_faults.loc[device_faults['time'].isin(df_[(df_[c] < vals[0]) | (df_[c] > vals[1])].time),'{}sensor'.format(col_names[c])]=c
        device_faults.loc[device_faults['time'].isin(df_[(df_[c] < vals[0]) | (df_[c] > vals[1])].time),'{}Faults'.format(col_names[c])]='{}WD'.format(col_names[c])
      else:
        print(c," has no wrong data points")
  else:
    print("No sensor data: ",df_.columns)
    
    #Plot analysis
  #Ploting correlation matrix between sensors
  cor=df_.corr()
  sns.heatmap(cor,annot=True, cmap="RdYlGn")
  
  i=0
  colors={0:'blue',1:'green',2:'orange',3:'red',4:'pink',5:'purple',6:'yellow',7:'black',8:'cyan'}
  # matplotlib.rcParams['figure.figsize'] = (15,8)
  fig, ax = plt.subplots(num_of_sensors,1,figsize=(25,40))

  if(len(df_.columns)>2):
    for c in df_.columns[2:]:
      ax[0].plot(df_["index"],df_[c],color=colors[i],label=c)
      i+=1
    ax[0].legend()
    ax[0].set_xlabel("index")
    ax[0].set_ylabel("Sensors")
    ax[0].set_title("Variation with index")


    plt.xticks(np.arange(0, len(df_["index"]) + 1, 50))
    # rotate the labels
    plt.xticks(rotation = 45)
  

 
    j=1
  
    for c in df_.columns[2:]:
      #Find local peaks
      df_['{}min'.format(col_names[c])] = df_[c][(df_[c].shift(1) > df_[c]) & (df_[c].shift(-1) > df_[c])]
      df_['{}max'.format(col_names[c])] = df_[c][(df_[c].shift(1) < df_[c]) & (df_[c].shift(-1) < df_[c])]
    
    print(df_.head())

    for c in df_.columns[2:num_of_sensors+1]:
      # Analysing variations
      if((df_[c]>df_[c].median()*1.2).any() or (df_[c]<df_[c].median()*0.8).any()):
        print("Time durations for ",c," variations: ")
        print(df_[(df_[c]>df_[c].median()*1.2)|(df_[c]<df_[c].median()*0.8)])
        device_variations=device_variations.append(df_[(df_[c]>df_[c].median()*1.2)|(df_[c]<df_[c].median()*0.8)])
        device_variations.loc[device_variations['time'].isin(df_[(df_[c]>df_[c].median()*1.2)|(df_[c]<df_[c].median()*0.8)].time),'{}sensor'.format(col_names[c])]=c
        device_variations.loc[device_variations['time'].isin(df_[(df_[c]>df_[c].median()*1.2)|(df_[c]<df_[c].median()*0.8)].time),'{}Faults'.format(col_names[c])]='{}VAR'.format(col_names[c])
      else:
        print(c," has no variation data points")

    for c in df_.columns[2:num_of_sensors+1]:
      ax[j].plot(df_["index"],df_[c],color='blue')
      ax[j].scatter(df_["index"], df_['{}min'.format(col_names[c])], c='r')
      ax[j].scatter(df_["index"], df_['{}max'.format(col_names[c])], c='g') 
      ax[j].set_title("{} vs index".format(c))
      ax[j].set_ylabel(c)
      ax[j].set_xlabel("index")
      j+=1
    # set the spacing between subplots
    # plt.subplots_adjust(left=0.1,bottom=0.9,right=0.9,top=0.9,wspace=0.4,hspace=0.4)
    plt.show()
  else:
    print("No sensor data: ",df_.columns)
  print(device_faults.head())
  print(device_variations.head())
  device_faults.to_csv("/content/drive/MyDrive/RCA/faults/"+str(names[num])+".csv",index = False)
  device_variations.to_csv("/content/drive/MyDrive/RCA/variations/"+str(names[num])+".csv",index = False)

In [ ]:
df_=create_device_data(names,dataframes,100)
auto_analyse(names,100,df_)

In [ ]:
for i in range(0,30):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(30,60):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
# for i in range(60,120):
#   df_=create_device_data(names,dataframes,i)
#   auto_analyse(names,i,df_)

In [ ]:
df_=create_device_data(names,dataframes,100)
auto_analyse(names,100,df_)

In [ ]:
for i in range(101,130):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(112,130):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(130,200):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(173,200):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(200,294):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(211,294):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(214,220):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
names.index('869170033894826')

219

In [ ]:
for i in range(220,250):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)

In [ ]:
for i in range(251,294):
  df_=create_device_data(names,dataframes,i)
  auto_analyse(names,i,df_)